<!-- ![](./header.png) -->
<img src="./header.png",width=100%>

# Selection of a performance metric for PLAsTiCC

*Alex Malz (NYU)*, *Tarek Alam (UCL)*, *Anita Bahmanyar (U. Toronto)*, *Rahul Biswas (U. Stockholm)*, *Renee Hlozek (U. Toronto)*, *Rafael Martinez-Galarza (Harvard)*, *Gautham Narayan (STScI)*

We describe and illustrate the process by which a global performance metric was chosen for Photometric LSST Astronomical Time-series Classification Challenge (PLAsTiCC), a Kaggle competition aiming to identify promising transient and variable classifiers for LSST by involving the broader community outside astronomy.

Introduction
============

The metric of this note is for the first version of the Kaggle competition, though there are future plans for an early classification challenge and identification of class-specific metrics for different science goals.

* The metric must return a single scalar value.
* The metric must be well-defined for non-binary classes.
* The metric must balance diverse science use cases in the presence of heavily nonuniform class prevalence.
* The metric must respect the information content of probabilistic classifications.
* The metric must be able to evaluate deterministic classifications.
* The metric must be interpretable, meaning it gives a more optimal value for "good" mock classifiers and a less optimal value for mock classifiers plagued by anticipated systematic errors; in other words, it must pass basic tests of intuition.
* The metric must be reliable, giving consistent results for different instantiations of the same test case.

Data
====

## Mock classifier systematics

* idealized: highly accurate on all classes
* guessing: random classifications across all classes
* tunnel vision: classifies one class well and others randomly
* cruise control: classifies all objects as a single class
* subsumed: consistently misclassifies one class as one other class

*show confusion matrices*

## Real classification results

* SNPhotCC \[from Michelle?\]
* \[Ashish's data?\]
* \[Renee's data?\]

*show confusion matrices*

Methods (Metrics)
======

* Brier score
* Log-loss

Results
=======

*one plot per set of "true" classes: classifiers on x axis, metrics on y axes*

Conclusions
===========

Write about your conclusions here. You have drawn some, right?